# SARIMA for Timeseries Forecasting
R is a strong language for performing statistical analysis, such as timeseries forecasting using the (S)ARIMA statistical model. SARIMA is powerful for forecasting time series data with seasonal cycles and trends, such as the weather, which we do in this notebook.